# Initial Mocap Data Exploration

Specifically a **subset** of the data used in D Holden's DLFCMS project, specifically **CMU Locomotion** categorized data.

This is after the pre-processing in python.

In [1]:
using LinearAlgebra, Statistics, Random
using NPZ
using PyPlot
using ArgCheck
using AxUtil, Flux
using ProgressMeter, Formatting

In [2]:
data_path = "../../../mocap-mtds/model_data/"
#= Load from Numpy Compressed files dumped from python pre-processing =#
Ys = npzread(joinpath(data_path, "Y_output_data.npz"))
Xs = npzread(joinpath(data_path, "X_input_data.npz"))
Ls = npzread(joinpath(data_path, "lengthsXY.npz"))

#= Extract from Dict --> Array of Matrices =#
N = maximum([parse(Int, x[1]) for x in match.(r"arr_([0-9]+)", keys(Ys))])
Ys = [Ys["arr_"*string(i)] for i in 0:N]
Xs = [Xs["arr_"*string(i)] for i in 0:N];

In [3]:
# X: trajectory(24), all joint pos(63), all joint rot(63)
# Y: root rvel, root xvel, root zvel, foots(4), all joint pos(63), all joint rot(63)

In [32]:
extract_joint_pos(y) = begin; @argcheck size(y,2) == 133; y[:,8:70]; end
extract_joint_rot(y) = begin; @argcheck size(y,2) == 133; y[:,71:133]; end
extract_trajectory(x) = begin; @argcheck size(x,2) == 150; x[:,1:48]; end

extract_trajectory (generic function with 1 method)

In [50]:
kwidentity(x; kwargs...) = identity(x)

kwidentity (generic function with 1 method)

In [ ]:
fig, axs = subplots(1,2,figsize=(9,4))
for i in 1:2
    f = (i == 1) ? kwidentity : diff
    axs[i].plot(f(extract_joint_pos(Ys[12])[:,9:9], dims=1), c=ColorMap("tab10")(i-1));
    (i == 2) && axs[i].axhline(0, linestyle=":");
    axs[i].set_title((i==1) ? "Original" : "Diff")
end

## Modelling asymmetric periodic behaviour

* Gradient of ascent $\ne$ gradient of descent.
* Time at peak $\ne$ time at trough

In [7]:
# Orthogonal RNN
mutable struct ORNNCell{F,A,V}
  σ::F
  Wi::A
  Whpar::A
  b::V
  h::V
  d::Int
end

ORNNCell(in::Integer, out::Integer, σ = tanh;
        init = Flux.glorot_uniform) =
  ORNNCell(σ, param(init(out, in)), param(zeros(Float32, Int(out*(out-1)/2), 1)),
          param(init(out)), param(ones(Float32, out)), out)

function (m::ORNNCell)(h, x)
  σ, Wi, b, d = m.σ, m.Wi, m.b, m.d
  Wh = AxUtil.Math.cayley_orthog(vec(m.Whpar)/10, d)
  h = σ.(Wi*x .+ Wh*h .+ b)
  return h, h
end

Flux.hidden(m::ORNNCell) = m.h

@Flux.treelike ORNNCell

function Base.show(io::IO, l::ORNNCell)
  print(io, "ORNNCell(", size(l.Wi, 2), ", ", size(l.Wi, 1))
  l.σ == identity || print(io, ", ", l.σ)
  print(io, ")")
end

"""
    ORNN(in::Integer, out::Integer, σ = tanh)
Orthogonal Recurrent Layer; like an RNN, but constraining the h->h W to be orthogonal
"""
ORNN(a...; ka...) = Flux.Recur(ORNNCell(a...; ka...))

ORNN

In [30]:
# Encoder-Orthogonal RNN
mutable struct EORNNCell{F,A,V}
  σ::F
  Whpar::A
  b::V
  h::V
  Wi::A
  encW1::A
  encW2::A
  encb1::V
  d::Int
end

EORNNCell(in::Integer, out::Integer, enc_h::Integer, σ = tanh;
        init = Flux.glorot_uniform) =
  EORNNCell(σ, param(zeros(Float32, Int(out*(out-1)/2), 1)),
          param(init(out)), param(ones(Float32, out)),     # Wh, b
          param(init(out, in)),                            # Wi
          param(init(enc_h, in)), param(init(out, enc_h)), # W1enc, W2enc
          param(init(enc_h)), out)                         # b1, d

function (m::EORNNCell)(h, x)
  σ, b, d, Wi, W1, W2, b1 = m.σ, m.b, m.d, m.Wi, m.encW1, m.encW2, m.encb1
  Wh = AxUtil.Math.cayley_orthog(vec(m.Whpar)/10, d)
  u = 0*W2*elu.(W1*x + b1)
  h = σ.(u + Wi*x + Wh*h .+ b)
  return h, h
end

Flux.hidden(m::EORNNCell) = m.h

@Flux.treelike EORNNCell

function Base.show(io::IO, l::EORNNCell)
  print(io, "EORNNCell(", size(l.encW1, 2), ", ", l.d, ", ", size(l.encW1, 1))
  print(io, ", ", l.σ)
  print(io, ")")
end

"""
    EORNN(in::Integer, out::Integer, σ = tanh)
Encoder-Orthogonal Recurrent Layer; like an Encoder-RNN, but constraining the h->h W to be orthogonal
"""
EORNN(a...; ka...) = Flux.Recur(EORNNCell(a...; ka...))

EORNN

In [212]:
n_ix = 50
d_out = 70
d_in = 48

d_state = 30
d_ff = 50
d_inpnn = 100
Y = Ys[n_ix][:,1:70];
T = size(Y,1)
X = [extract_trajectory(Xs[n_ix])[i,:] for i in 1:T];

diffdmodel = 0  #63

0

In [266]:
nn = ORNN(d_in, d_state)
ffnn = Chain(Dense(d_state, d_ff, elu), Dense(d_ff, d_out, identity))
B, C = param(Flux.glorot_uniform(d_out, d_in+diffdmodel)), param(Flux.glorot_uniform(d_out, d_state))
opt = ADAM(1e-2)
pars = Flux.params(Flux.params(nn)..., Flux.params(ffnn), B, C)

Params([Float32[-0.203467 -0.249476 … -0.160145 -0.195034; 0.00412299 -0.257373 … -0.0821489 0.0984335; … ; 0.0928166 0.0518591 … -0.0719532 -0.196095; 0.131831 -0.0885061 … 0.170789 0.176482] (tracked), Float32[0.0; 0.0; … ; 0.0; 0.0] (tracked), Float32[-0.143349, 0.330343, -0.119529, -0.0363682, 0.150469, 0.0838409, -0.341127, -0.435089, -0.435098, 0.0495103  …  0.0296094, -0.269639, 0.431959, 0.175207, -0.145179, -0.0504593, 0.44665, -0.113409, 0.0770207, -0.222874] (tracked), Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] (tracked), Float32[0.205727 0.0580389 … -0.131728 0.131469; -0.195477 -0.169232 … 0.217095 0.157328; … ; 0.201396 0.19142 … -0.0284474 0.199977; -0.159107 0.0232053 … -0.0678401 0.0378097] (tracked), Float32[0.211283 0.220935 … 0.106367 -0.0364616; -0.0416726 0.0245891 … 0.0139192 0.104493; … ; 0.132027 0.211125 … 0.219848 -0.185159; -0.161462 -0.0706165 … -0.0359056 -0.0665933] (tracked)])

In [241]:
# nn = EORNN(d_in, d_state, d_inpnn)
# ffnn = Chain(Dense(d_state, d_ff, elu), Dense(d_ff, d_out, identity))
# B, C = param(Flux.glorot_uniform(d_out, d_in+diffdmodel)), param(Flux.glorot_uniform(d_out, d_state))
# opt = ADAM(1e-2)
# pars = Flux.params(Flux.params(nn)..., Flux.params(ffnn), B, C)

## Multiple time series: joint optimisation

In [267]:
n_ixs = [12,50,100, 200, 300, 400, 500, 600, 700]
d_out = 70
d_in = 48

d_state = 30
d_ff = 50
d_inpnn = 100
Y = [Ys[n_ix][:,1:70] for n_ix in n_ixs];
T = [size(y,1) for y in Y]
X = [[extract_trajectory(Xs[n_ix])[i,:] for i in 1:T[i]] for (i,n_ix) in enumerate(n_ixs)];

hs = let init=Flux.glorot_uniform(d_state); [param(init) for n_ix in n_ixs]; end
pars = Flux.params(Flux.params(nn)..., Flux.params(ffnn)..., B, C, hs...)

diffdmodel = 0  #63

0

In [282]:
nepochs = 500
history = zeros(Float32, nepochs)
@showprogress for ee in 1:nepochs
    (ee % 100 == 0) && printfmtln("Epoch {:d}, cost {:.2f}", ee, history[ee-1]/length(n_ixs))
    for (i,n_ix) in enumerate(n_ixs)
        cX, cY = X[i], Y[i]
        nn.state = hs[i]
        x̂ = nn.(cX)
        x̂ = Tracker.collect(reduce(hcat, x̂))
        ŷ = (ffnn(x̂) + C*x̂ + B*Xs[n_ix][:,1:48+diffdmodel]')'
        obj = sum(x->x^2, cY - ŷ)/size(cY,2)

        history[ee] += obj.data
        Tracker.back!(obj)
        for p in pars
            Tracker.update!(opt, p, -Tracker.grad(p))
        end

        Flux.reset!(nn)
    end
end

Progress:  20%|████████                                 |  ETA: 0:06:41

Epoch 100, cost 7.36


Progress:  40%|████████████████▎                        |  ETA: 0:04:55

Epoch 200, cost 7.25


Progress:  60%|████████████████████████▍                |  ETA: 0:03:22

...

Progress:  80%|████████████████████████████████▋        |  ETA: 0:01:42

Epoch 400, cost 7.11


Progress: 100%|████████████████████████████████████████▉|  ETA: 0:00:02

Epoch 500, cost 6.86


Progress: 100%|█████████████████████████████████████████| Time: 0:08:35


In [ ]:
plot(history/length(n_ixs))
let m=minimum(history/length(n_ixs)); cl=gca().get_ylim(); gca().set_ylim(max(cl[1], 0.9*m), min(cl[2], 4*m)); end

In [281]:
opt.eta /= 2

0.00015625

In [305]:
ix = 7
x̂ = nn.(X[ix])
x̂ = Tracker.collect(reduce(hcat, x̂))
ŷ = (ffnn(x̂) + C*x̂ + B*Xs[n_ixs[ix]][:,1:48+diffdmodel]')'
Flux.reset!(nn)

In [622]:
fig, axs = subplots(6,4,figsize=(10,10))
offset = 0
for i in 1:6, j in 1:4
    axs[i,j].plot(Y[ix][:,offset+(i-1)*4 + j])
    axs[i,j].plot(ŷ.data[:,offset+(i-1)*4 + j])
end

UndefVarError: UndefVarError: Y not defined

## [ABOVE] It appears that the net can make inferences about style from the input vector.

Perhaps it is not surprising: the input vector quite clearly encodes something of the movement of the central joint. We therefore either need to smooth out the input vector (except smoothing will smooth out signal too -- think corners, perhaps we can use splines and robust regression?), or abandon the dataset in favour of a different one.

Enter Ian Mason's Style Database:

In [38]:
data = npzread("../mason/style_database.npz");

In [688]:
Xs, Ys, Ps = data["Xin"], data["Yin"], data["Pin"];

--------------------
### Create figures for proposal

Arguing that nonlinear models are required, and joint *rotations* do not look easier to model than joint *positions*.

1. Position vs Rotation (incl. diffs and spectrogram) (dim 11, joint 4.y)
2. Example assymetric periodic behaviour (joint 15, joint 5.z)

Note that (1.) also exemplifies assymetric periodic behaviour, but I thought it was easier to have a smaller chart in the main body

In [616]:

using FFTW, FFTViews, DSP
tmppos = Ys[1:500,used_joint_ixs .+ (28)];
tmprot = Ys[1:500,used_joint_ixs .+ (28 + 93*3)];

pgramplot(ax, x, c=t10b) = begin; pp=periodogram(x, fs=60); ax.plot(pp.freq, pp.power, c=c); 
    ax.set_title(format("{:.2f}", pp.freq[argmax(pp.power)])); end
t10o = ColorMap("tab10")(1);
t10b = ColorMap("tab10")(0);

# >> Chart 1. Pos vs Rotation (3 x 2 grid)
ii = 11
fig, axs = subplots(3,2,figsize=(12,14))
axs[1,1].plot(tmppos[:,ii]); axs[2,1].plot(diff(tmppos[:,ii]))
axs[1,2].plot(tmprot[:,ii], c=t10o); axs[2,2].plot(diff(tmprot[:,ii]), c=t10o)
axs[1,1].set_title("Position"); axs[2,1].set_title("Position Diff")
axs[1,2].set_title("Rotation"); axs[2,2].set_title("Rotation Diff")
pgramplot(axs[3,1], diff(tmppos[:,ii])); axs[3,1].set_title("PosDiff Spectrogram");
pgramplot(axs[3,2], diff(tmprot[:,ii]), t10o); axs[3,2].set_title("RotDiff Spectrogram");

[axs[i,j].set_xlabel("time") for i in 1:2, j in 1:2]
[axs[3,j].set_xlabel("freq") for j in 1:2];
tight_layout()
# savefig("joint4y_posrot.pdf")

# >> Chart 2. Assymetric period (1 x 2 grid)
ii = 15
fig, axs = subplots(1,2,figsize=(10,3))
axs[1].plot(identity(tmppos[1:300,ii]));  axs[1].set_title(L"Joint 5 $z$ position")
axs[2].plot(diff(tmppos[1:300,ii])); axs[2].set_title(L"Joint 5 $z$ velocity")
axs[2].axhline(0, linestyle=":", color="gray")
[axs[i].set_xlabel("time") for i in 1:2]
tight_layout()
# savefig("joint5z.pdf")

-----------------
and continue with the main event.....

In [689]:
# extract joint positions 31 => 21 joints à la Holden original CMU code
used_joints = [0,2,3,4,5,7,8,9,10,12,13,15,16,18,19,20,22,25,26,27,29] .+ 1
used_joint_ixs = vec(reshape(1:93,3,31)[:,used_joints]);

Xs = Xs[:,1:48]  # trajectory only
Ys = Ys[:,[1,2,3, (used_joint_ixs .+ 28)...]];

In [368]:
""" Slide over windows """
function windowfy(X, Y; window=240, window_step=120)
    n = size(X, 1)
    @argcheck size(X, 1) == size(Y, 1)
    
    windowsY = []
    windowsX = []
    if n >= 50
        for j in range(1, stop=max(n-window_step-10,1), step=window_step)
            append!(windowsY, Y[j:j+window,:])
            append!(windowsX, X[j:j+window,:])
        end
    end
    return windowsX, windowsY
end

windowfy

In [668]:
# difference the (input) trajectories so they each have equivalent units / ranges
@argcheck size(Xs, 2) == 48
Xs = hcat(diff(Xs[:,1:12], dims=2), diff(Xs[:,13:24], dims=2), diff(Xs[:,25:36], dims=2),
      diff(Xs[:,37:48], dims=2));

In [669]:
if true || !all(isapprox.(mean(Xs, dims=1), 0.0, atol=1e-4))
    Xmean, Xstd = mean(Xs, dims=1), std(Xs, dims=1)
    Ymean, Ystd = mean(Ys, dims=1), std(Ys, dims=1)
end

(Float32[0.000851498 0.0226859 … 1.4899 0.255066], Float32[0.00882783 0.0120419 … 0.273567 0.28317])

In [670]:
Xs = Xs ./ mean(Xstd)
Ys = (Ys .- Ymean) ./ Ystd;

In [671]:
# add in phase to inputs
Ps_trig = hcat(sin.(Ps .* 2π), cos.(Ps .* 2π));
Xs = hcat(Xs, Ps_trig);

 ## Optimisation

In [10]:
d_out = 66
d_in = 48 + 2
d_state = 30
d_ff = 50
diffdmodel = 0  #63

nn = RNN(d_in, d_state, elu)
ffnn = Chain(Dense(d_state, d_ff), Dense(d_ff, d_out, identity))

Whpar = param(zeros(Float32, Int(d_state*(d_state-1)/2)))
B, C = param(Flux.glorot_uniform(d_out, d_in+diffdmodel)), param(Flux.glorot_uniform(d_out, d_state))
h0 = param(rand(Float32, d_state))

opt = ADAM(1e-3)

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [11]:
n_ix = 120000 #[500, 22500, 120000]

Y = Ys[n_ix:n_ix+199,:];
T = size(Y,1)
X = Xs[n_ix:n_ix+199,:]
Xlist = [X[i,:] for i in 1:T];

pars = Flux.params(Flux.params(nn)..., Flux.params(ffnn), B, C, Whpar);

In [12]:
n_ixs = Int.(collect(range(10000, stop=120000, length=101)))
n_ixs = Int.(collect(range(1, stop=20000, step=200)))
Y = [Ys[n_ix:n_ix+199,:] for n_ix in n_ixs];
T = [size(y,1) for y in Y]
X = [Xs[n_ix:n_ix+199,:] for n_ix in n_ixs];
Xlists = [[x[i,:] for i in 1:T[i]] for (i,x) in enumerate(X)];

hs =  param(zeros(Float32, length(n_ixs)))
pars = Flux.params(Flux.params(nn)..., Flux.params(ffnn)..., B, C, Whpar);

In [13]:
function zero_grad!(P) 
    for x in P
        x.grad .= 0
    end
end
zero_grad!(pars)
Flux.reset!(nn)
Flux.truncate!(nn)

In [14]:
nepochs = 100
history = zeros(Float32, nepochs)
batches = let N=length(X); P=randperm(N); [reshape(P, Int(N/5), 5)[i,:] for i in 1:Int(N/5)]; end

@showprogress for ee in 1:nepochs
    (ee % 5 == 0) && printfmtln("Epoch {:d}, cost {:.2f}", ee, history[ee-1]/length(n_ixs))
    for b in batches
        for i in b
            cX, cXlist, cY = X[i], Xlists[i], Y[i]
            nn.cell.Wh = AxUtil.Math.cayley_orthog(Whpar/10, d_state)
            x̂ = nn.(cXlist)
            x̂ = Tracker.collect(reduce(hcat, x̂))
            ŷ = (ffnn(x̂) + C*x̂ + B*cX')'
            obj = sum(x->x^2, cY - ŷ)/size(cY,2)
            Tracker.back!(obj)
            
            history[ee] += obj.data
            Flux.reset!(nn)
        end
        for p in pars
            Tracker.update!(opt, p, -Tracker.grad(p)/length(b))
        end
    end
end

UndefVarError: UndefVarError: randperm not defined

In [ ]:
plot(history/length(n_ixs))
let m=minimum(history[history.>0]/length(n_ixs)); 
    cl=gca().get_ylim(); gca().set_ylim(max(cl[1], 0.9*m), min(cl[2], 4*m)); end

In [1011]:
opt.eta *= 1.5

0.0015

In [998]:
ix = 2
nn.state = ((nn.state .+ tanh.(hs[ix]) .+ 1) .% 2) .- 1  
x̂ = nn.(Xlists[ix])
x̂ = Tracker.collect(reduce(hcat, x̂))
ŷ = (ffnn(x̂) + C*x̂ + B*X[ix]')'
Flux.reset!(nn)
sum(x->x^2, Y[ix] - ŷ)/size(Y[ix],2)

42.38097274795309 (tracked)

In [ ]:
fig, axs = subplots(6,4,figsize=(10,10))
offset = 24
for i in 1:6, j in 1:4
    axs[i,j].plot(Y[ix][:,offset+(i-1)*4 + j])
    axs[i,j].plot(ŷ.data[:,offset+(i-1)*4 + j])
end

In [1014]:
ii = 1

1

In [ ]:
fig, axs = subplots(1,2,figsize=(10,5))
traj_rec = hcat(vcat(-sum(Xs[ii,1:6]), cumsum(Xs[ii,1:11]) .- sum(Xs[ii,1:6])), 
                vcat(-sum(Xs[ii,12:17]), cumsum(Xs[ii,12:22]) .- sum(Xs[ii,12:17])))
axs[1].scatter(traj_rec[:,1], traj_rec[:,2])
[axs[1].arrow(traj_rec[i], traj_rec[12+i], Xs[ii,i]*0.93, Xs[ii,i+11]*0.93, 
           head_width=.02, head_length=0.04, length_includes_head=true, color="black") for i in 1:11];
axs[1].scatter(0,0,color="orange")
axs[1].set_title(ii)

[axs[2].arrow(0, 0, Xs[ii,i]*0.93, Xs[ii,i+11]*0.93, 
           head_width=.02, head_length=0.04, length_includes_head=true, color=ColorMap("cool")(i/11)) for i in 1:11];
[axs[2].text(Xs[ii,i]*0.93, Xs[ii,i+11]*0.93, i) for i in 1:11];
axs[2].set_ylim(-0.7,0.7)
axs[2].set_xlim(-0.5,0.5)
ii += 3

In [561]:
@benchmark begin;
    nn.state = hs[ix]
    x̂ = nn.(Xlists[ix])
    x̂ = Tracker.collect(reduce(hcat, x̂))
    ŷ = (ffnn(x̂) + C*x̂ + B*X[ix]')'
    Tracker.back!(sum(x->x^2, ŷ - Y[ix]))
    Flux.reset!(nn)
end

BenchmarkTools.Trial: 
  memory estimate:  26.31 MiB
  allocs estimate:  160937
  --------------
  minimum time:     79.986 ms (15.10% GC)
  median time:      100.524 ms (17.51% GC)
  mean time:        109.110 ms (16.93% GC)
  maximum time:     151.210 ms (22.95% GC)
  --------------
  samples:          46
  evals/sample:     1

In [562]:
nn2 = RNN(d_in, d_state)

Recur(RNNCell(48, 30, tanh))

In [565]:
@benchmark begin;
    Q = AxUtil.Math.cayley_orthog(vec(nn.cell.Whpar)/10, d_state)
    nn2.cell.Wh = Q
    nn2.state = hs[ix]
    x̂ = nn2.(Xlists[ix])
    x̂ = Tracker.collect(reduce(hcat, x̂))
    ŷ = (ffnn(x̂) + C*x̂ + B*X[ix]')'
    Tracker.back!(sum(x->x^2, ŷ - Y[ix]))
    Flux.reset!(nn2)
end

BenchmarkTools.Trial: 
  memory estimate:  6.76 MiB
  allocs estimate:  81734
  --------------
  minimum time:     13.490 ms (0.00% GC)
  median time:      14.764 ms (0.00% GC)
  mean time:        19.289 ms (24.44% GC)
  maximum time:     42.484 ms (55.10% GC)
  --------------
  samples:          260
  evals/sample:     1

# Visualisation

In [672]:
using Quaternions  # <= still need
using MeshCat # <= still need
# using CoordinateTransformations
# using GeometryTypes
# using Colors: RGBA, RGB

### Visualise CMU Data

In [2]:
data_path = "../../../mocap-mtds/model_data/"
loadYmu = npzread(joinpath(data_path, "Ymean.npz"))["Ymean"];
loadYstd = npzread(joinpath(data_path, "Ystd.npz"))["Ystd"];

In [3]:
YsCMU = npzread(joinpath(data_path, "Y_output_data.npz"))
YsCMU = [YsCMU["arr_"*string(i-1)] for i in 1:length(YsCMU)];

In [713]:
YsAngry = npzread(joinpath(data_path, "Y_angry_01.npz"))

Dict{String,Any} with 1 entry:
  "Ys" => Float32[-0.0229391 -0.0223341 … 0.0362696 0.0365848]…

In [236]:
qimag = Quaternions.imag
quat_list(x) = [quat(x[i,:]) for i in 1:size(x,1)]
quat_list_to_mat(x) = reduce(vcat, [qimag(xx)' for xx in x])
quaterion_angle_axis_w_y(θ) = quat(cos(θ/2), 0, sin(θ/2), 0)
apply_rotation(x, qrot) = qrot * x * conj(qrot)

function reconstruct_positions_old(Y::Matrix; denormalize=true)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    if denormalize
        Y = Y .* loadYstd' .+ loadYmu'
    end
    
    n = size(Y, 1)
    root_r, root_x, root_z, joints = Y[:,1], Y[:,2], Y[:,3], Y[:,8:(63+7)]
    joints = reshape(joints, n, 3, 21)
    joints = permutedims(joints, [1,3,2])
    rotation = Quaternion(1.0)
    offsets = []
    translation = zeros(3)

    for i = 1:n
        joints[i,:,:] = apply_rotation(quat_list(joints[i,:,:]), rotation) |> quat_list_to_mat
        joints[i,:,1] = joints[i,:,1] .+ translation[1]
        joints[i,:,3] = joints[i,:,3] .+ translation[3]
        
        rotation = quaterion_angle_axis_w_y(-root_r[i]) * rotation
        append!(offsets, apply_rotation(quat(0.,0,0,1), rotation))
        translation = translation + qimag(apply_rotation(quat(0., root_x[i], 0, root_z[i]), rotation))
    end
    
    return joints
end

function reconstruct_positions(Y::Matrix; denormalize=true)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    if denormalize
        Y = Y .* loadYstd' .+ loadYmu'
    end
    
    root_r, root_x, root_z, joints = Y[:,1], Y[:,2], Y[:,3], Y[:,8:(63+7)]
    return _joints_fk(joints, root_x, root_z, root_r)
end

function _joints_fk(joints::Matrix{T}, root_x::Vector{T}, root_z::Vector{T}, 
        root_r::Vector{T}) where T <: Number

    n = size(joints, 1)
    joints = reshape(joints, n, 3, 21)
    joints = permutedims(joints, [1,3,2])
    rotation = Quaternion(1.0)
    offsets = []
    translation = zeros(3)

    for i = 1:n
        joints[i,:,:] = apply_rotation(quat_list(joints[i,:,:]), rotation) |> quat_list_to_mat
        joints[i,:,1] = joints[i,:,1] .+ translation[1]
        joints[i,:,3] = joints[i,:,3] .+ translation[3]
        
        rotation = quaterion_angle_axis_w_y(-root_r[i]) * rotation
        append!(offsets, apply_rotation(quat(0.,0,0,1), rotation))
        translation = translation + qimag(apply_rotation(quat(0., root_x[i], 0, root_z[i]), rotation))
    end
    
    return joints
end

_joints_fk (generic function with 1 method)

In [719]:
angry_ok = reconstruct_positions(YsAngry["Ys"][1,:,:], denormalize=false);

In [722]:
vis = mocapviz.create_animation([angry_ok[1:300,:,:]], "angry_ok"; vis=vis, linemesh=[mocapviz.redmesh])

MeshCat Visualizer with path /meshcat

In [644]:
recon, recon2 = reconstruct_positions(YsCMU[13]), reconstruct_positions(YsCMU[18]);

In [ ]:
scatter(recon[1,:,1], recon[1,:,2])
[plt.text(recon[1,i,1], recon[1,i,2], i) for i in 1:length(recon[1,:,1])];

In [11]:
# Create a new visualizer instance
vis = Visualizer()
open(vis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8700
└ @ MeshCat /Users/alexbird/.julia/dev/MeshCat/src/servers.jl:24


Process(`open http://127.0.0.1:8700`, ProcessExited(0))

┌ Error: error handling request
│   exception = (Base.IOError("stream is closed or unusable", 0), Base.StackTraces.StackFrame[check_open at stream.jl:323 [inlined], uv_write_async(::TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:871, uv_write(::TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:845, unsafe_write(::TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:901, macro expansion at ConnectionPool.jl:132 [inlined], write at io.jl:165 [inlined], closebody at Streams.jl:111 [inlined], closewrite(::Stream{Request,Transaction{TCPSocket}}) at Streams.jl:126, (::##13#14{#_servercoroutine#11{ServerWS},Transaction{TCPSocket},Stream{Request,Transaction{TCPSocket}}})() at task.jl:259])
└ @ HTTP.Servers /Users/alexbird/.julia/packages/HTTP/GN0Te/src/Servers.jl:356


In [192]:
include("../../../mocap-mtds/mocap_viz.jl")

Main.mocapviz

In [193]:
vis = mocapviz.create_animation([recon, recon2], "test"; vis=vis, linemesh=[mocapviz.yellowmesh, mocapviz.redmesh])

MeshCat Visualizer with path /meshcat

In [ ]:
delete!(vis)

### Visualise Xia et al. Data

In [687]:
Ys[:,3]

123426-element Array{Float32,1}:
  0.097789206
  0.019930221
 -0.104543135
 -0.1940918  
 -0.2679099  
 -0.31946385 
 -0.3424955  
 -0.33781835 
 -0.30423185 
 -0.24853864 
 -0.18442912 
 -0.12457695 
 -0.07718195 
  ⋮          
  0.96893865 
  0.9441292  
  0.9050325  
  0.850461   
  0.7819367  
  0.70069945 
  0.6146078  
  0.5278602  
  0.44243118 
  0.36326677 
  0.29459813 
  0.23552004 

In [701]:
# function reconstruct_positions_xia(Y::Matrix; denormalize=true)
#     Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
#     if denormalize
#         Y = Y .* Ystd .+ Ymean
#     end
#     _joints_fk(Y[:, 4:66], Y[:,1], Y[:,2], vcat(0, Y[:,3])*20*π/180)
# end

function reconstruct_positions_xia(Y::Matrix; denormalize=true)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    if denormalize
        Y = Y .* Ystd .+ Ymean
    end
    
    n = size(Y, 1)
    root_x, root_z, root_r, joints = Y[:,1], Y[:,2], Y[:,3]*40*π/180, Y[:,4:(63+3)]
    joints = reshape(joints, n, 3, 21)
    joints = permutedims(joints, [1,3,2])
    rotation = Quaternion(1.0)
    offsets = []
    translation = zeros(3)

    for i = 1:n
        joints[i,:,:] = apply_rotation(quat_list(joints[i,:,:]), rotation) |> quat_list_to_mat
        joints[i,:,1] = joints[i,:,1] .+ translation[1]
        joints[i,:,3] = joints[i,:,3] .+ translation[3]
        
        rotation = quaterion_angle_axis_w_y(-root_r[i])
        append!(offsets, apply_rotation(quat(0.,0,0,1), rotation))
        translation = translation + qimag(apply_rotation(quat(0., root_x[i], 0, root_z[i]), rotation))
    end
    
    return joints
end

reconstruct_positions_xia (generic function with 1 method)

In [723]:
# No normalization applied
Xs, Ys, Ps = data["Xin"], data["Yin"], data["Pin"];
Xs = Xs[:,1:48]  # trajectory only
Ys = Ys[:,[1,2,3, (used_joint_ixs .+ 28)...]];

In [ ]:
xrecon1 = reconstruct_positions_xia(Ys[1:300, :], denormalize=false);
# vis = mocapviz.create_animation([xrecon1], "mason"; vis=vis, linemesh=mocapviz.yellowmesh, scale=1.0)

In [ ]:
vis = mocapviz.create_animation([xrecon1[1:120,:,:]*10, angry_ok[1:120,:,:]], "angry_ok"; vis=vis, 
    linemesh=[mocapviz.yellowmesh, mocapviz.redmesh])

In [ ]:
plot(angry_ok[range(ii,step=10,length=12), 1,1], angry_ok[range(ii,step=10,length=12), 1,3])
ii+=10

In [732]:
vis = mocapviz.create_animation([xrecon1[1:270,:,:]*10, angry_ok[31:300,:,:]], "angry_ok"; vis=vis, 
    linemesh=[mocapviz.yellowmesh, mocapviz.redmesh])

MeshCat Visualizer with path /meshcat

In [726]:
delete!(vis)

MeshCat Visualizer with path /meshcat

In [347]:
npzwrite("juliaYinp.npz", Ys[1:300, :])

In [365]:
npzwrite("juliaxia1.npz", xrecon1)

In [118]:
testY = data["Yin"][1:2000,[1,2,3, (used_joint_ixs .+ 28)...]];
testY = Ys[1:2000,4:end];

In [119]:
testYrs = permutedims(reshape(testY, 2000, 3, 21), [1,3,2]);

In [170]:
vis = mocapviz.create_animation([xrecon1], "mason"; vis=vis, linemesh=mocapviz.yellowmesh, scale=1.0)

MeshCat Visualizer with path /meshcat

In [ ]:
reconstruct_positions(YsCMU[18]);

In [27]:
setobject!(vis[:line1][:line_1, ], HyperRectangle(Vec3(0.), Vec3([1.,0.03,0.03])), blackmesh)

MeshCat Visualizer with path /meshcat/line1/line_1

In [ ]:
lines = LineCollection([HyperRectangle(zero(Point{3, Float64}), Point{3}([1.,0,0]), 0.02) 
                        for i in 1:nl], [linestr(i) for i in 1:nl], [yellowmesh for i in 1:nl])

In [20]:
using ColorTypes: RGBA
using GeometryTypes
blackmesh = MeshPhongMaterial(color=RGBA(0.1, 0.1, 0.1, 0.7))

MeshCat.GenericMaterial
  _type: String "MeshPhongMaterial"
  color: RGBA{Float32}
  map: Nothing nothing
  depthFunc: Int64 3
  depthTest: Bool true
  depthWrite: Bool true
  vertexColors: Int64 0
  side: Int64 2


## Moved this out into mocap_viz.jl

So this is here momentarily in case I messed something up.

In [ ]:
dotstr(i) = "dot_" * format("{:02d}", i);
linestr(i) = "line_" * format("{:02d}", i);

greymesh = MeshPhongMaterial(color=RGBA(0.3, 0.3, 0.3, 0.7))
yellowmesh = MeshPhongMaterial(color=RGBA(204/255, 204/255, 0., 0.7));
redmesh = MeshPhongMaterial(color=RGBA(224/255, 131/255, 94/255, 0.7));

# initialise lines and dots
mutable struct LineCollection{T, R}
    data::Array{T}
    vnames::Array{String}
    attributes::Array{R}
end
mutable struct JointCollection{T, R}
    data::Array{T}
    vnames::Array{String}
    attributes::Array{R}
end

Base.length(x::LineCollection) = length(x.data)
Base.length(x::JointCollection) = length(x.data)

function update_pos!(lines::LineCollection, origin::Matrix, extremity::Matrix)
    @argcheck length(lines) == size(origin, 1) == size(extremity, 1)
    lines.data = [Cylinder(Point{3}(origin[i,:]), Point{3}(extremity[i,:]), l.r)
                  for (i,l) in enumerate(lines.data)]
end

function update_pos!(lines::LineCollection, data::Matrix)
    parents = [1,2,3,4,1,6,7,8,1,10,11,12,12,14,15,16,12,18,19,20][1:length(lines)]
    parent = data[parents,:]
    data = data[2:end, :]  # remove root (has no line)
    update_pos!(lines, parent, data)
end

function update_pos!(joints::JointCollection, data::Matrix)
    @argcheck length(joints) == size(data, 1)
    joints.data = [HyperSphere(Point3f0(data[i,:]), l.r) for (i,l) in enumerate(joints.data)]
end

# 3D plotting utilities: map line from x₁ → x₂ into affine transformation of e₁.
function line_into_transform(x1, x2)
    scaling, position, rotation = line_into_transforms_indiv(x1, x2)
    return AffineMap(rotation, position) ∘ LinearMap(diagm(0=>scaling))
end

function line_into_transforms_indiv(x1, x2)
    scaling = [norm(x2-x1), 1., 1]
    position = x1
    rotation = CoordinateTransformations.rotation_between([1., 0, 0], x2 - x1)
    return scaling, position, rotation
end

line_into_transform(x::Cylinder) = line_into_transform(x.origin, x.extremity)
line_into_transforms_indiv(x::Cylinder) = line_into_transforms_indiv(x.origin, x.extremity)

function setobj_mocap!(v::AbstractVisualizer, L::LineCollection, J::JointCollection)
    # lines at back, joints at front
    for i in 1:length(L)
        setobject!(v[L.vnames[i]], L.data[i], L.attributes[i])
    end
    for i in 1:length(J)
        setobject!(v[J.vnames[i]], J.data[i], J.attributes[i])
    end
end

function settransform_mocap!(v::AbstractVisualizer, L::LineCollection, J::JointCollection, anim=false)
    # lines at back, joints at front
    if anim
        for i in 1:length(L)
            scaling, position, rotation = line_into_transforms_indiv(L.data[i])
            anim_settransform!(v[L.vnames[i]], scaling, position, rotation)
        end
    else
        for i in 1:length(L)
            tform = line_into_transform(L.data[i])
            settransform!(v[L.vnames[i]], tform)
        end
    end
    for i in 1:length(J)
        settransform!(v[J.vnames[i]], Translation(J.data[i].center))
    end
end

nj = 21
nl = nj-1
lines = LineCollection([Cylinder(zero(Point{3, Float64}), Point{3}([1.,0,0]), 0.02) 
                        for i in 1:nl], [linestr(i) for i in 1:nl], [yellowmesh for i in 1:nl])
joints = JointCollection([HyperSphere(Point3f0(0), 0.05f0) for i in 1:nj],
                        [dotstr(i) for i in 1:nj], [greymesh for i in 1:nj]);

setobj_mocap!(vis[:skeleton], lines, joints)

scale_α = 0.1
ii = 1
# need to permute z/y axis for Blink/Three.js setup
update_pos!(lines, recon[ii,1:nj,[1,3,2]]*scale_α)
update_pos!(joints, recon[ii,1:nj,[1,3,2]]*scale_α);
settransform_mocap!(vis[:skeleton], lines, joints)

function anim_settransform!(vis::MeshCat.AnimationFrameVisualizer, scaling, position, rotation)
    clip = MeshCat.getclip!(vis)
    MeshCat._setprop!(clip, vis.current_frame, "scale", "vector3", scaling)
    MeshCat._setprop!(clip, vis.current_frame, "position", "vector3", position)
    MeshCat._setprop!(clip, vis.current_frame, "quaternion", "quaternion", MeshCat.js_quaternion(rotation))
end

anim = Animation()

scale_α = 0.1
# need to permute z/y axis for three.js setup


for ii in 1:240
    atframe(anim, vis, ii-1) do frame
        update_pos!(lines, recon[ii,1:nj,[1,3,2]]*scale_α)
        update_pos!(joints, recon[ii,1:nj,[1,3,2]]*scale_α);
        settransform_mocap!(frame[:skeleton], lines, joints, true)
    end
end


# `setanimation!()` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
setanimation!(vis, anim)